In [1]:
import sys
import os

# This means you will use the first GPU among the four GPUs in our case.
# "0", "1", "2", "3". Since FT dataset is small, using one GPU should be proper.
os.environ["CUDA_VISIBLE_DEVICES"]= "0"


import torch
import numpy as np
import pandas as pd
import warnings
import lightning as L
torch.set_float32_matmul_precision('high')

# Filter out FutureWarning and UnderReviewWarning messages from pl_bolts
warnings.filterwarnings("ignore", module="pl_bolts")

# Add the parent directory to sys.path
sys.path.append( '../')

from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
import tokenizer_sol, datamodule_finetune_sol, model_finetune_sol, chemllama_mtr, utils_sol
import auto_evaluator_sol

# print(os.path.dirname(__file__))


torch.manual_seed(1004)
np.random.seed(1004)

print(os.getcwd())

2024-06-18 18:07:26.154382: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 18:07:26.190265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 18:07:26.190290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 18:07:26.190311: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-18 18:07:26.196796: I tensorflow/core/platform/cpu_feature_g

/home/ylee/SolLlama


/home/ylee/chemllm/lib/python3.11/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/ylee/chemllm/lib/python3.11/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:92: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiveTask]

In [2]:
# Clone the pretrained model's repository
# utils_sol.get_pretrained_model()

In [3]:
#### Hyper Parameters ##### <- You can control these parameters as you want
# solute_or_solvent = 'solvent'
solute_or_solvent = 'solute'
ver_ft = 0 # version control for FT model & evaluation data # Or it will overwrite the models and results
batch_size_pair = [64, 64] if solute_or_solvent == 'solute' else [10, 10] # [train, valid(test)] 
# since 'solute' has very small dataset. So I thinl 10 for train and 10 for valid(test) should be the maximum values.
lr = 0.0001 
epochs = 7
use_freeze = False  # Freeze the model or not # False measn not freezing
overwrite_level_2 = True # If you don't want to overwrite the models and csv files, then change this to False
###########################

In [4]:
max_seq_length = 512
tokenizer = tokenizer_sol.fn_load_tokenizer_llama(
    max_seq_length=max_seq_length,
)
max_length = max_seq_length
num_workers = 2

In [5]:
################################
ep = 6 # this is the target epoch to load the trained fine-tuned model
# dir_main = "/home/ylee/SolLlama"
dir_main = "./"
#################################
name_model_mtr = "ChemLlama_Medium_30m_vloss_val_loss=0.029_ep_epoch=04.ckpt" 

dir_model_mtr = f"{dir_main}/model_mtr/{name_model_mtr}"

ver_ft = 0
dir_model_ft_to_save = f"{dir_main}/save_models_ft/ft_version_{ver_ft}"
name_model_ft = f'AbraLlama_{solute_or_solvent}'

In [6]:
# Load dataset for finetune
batch_size_for_train = batch_size_pair[0]
batch_size_for_valid = batch_size_pair[1]

data_module = datamodule_finetune_sol.CustomFinetuneDataModule(
    solute_or_solvent=solute_or_solvent,
    tokenizer=tokenizer,
    max_seq_length=max_length,
    batch_size_train=batch_size_for_train,
    batch_size_valid=batch_size_for_valid,
    # num_device=int(config.NUM_DEVICE) * config.NUM_WORKERS_MULTIPLIER,
    num_device=num_workers,
)

data_module.prepare_data()
data_module.setup()
steps_per_epoch = len(data_module.train_dataloader())

# Load model and optimizer for finetune
learning_rate = lr


model_mtr = chemllama_mtr.ChemLlama.load_from_checkpoint(dir_model_mtr)


model_ft = model_finetune_sol.CustomFinetuneModel(
    model_mtr=model_mtr,
    steps_per_epoch=steps_per_epoch,
    warmup_epochs=1,
    max_epochs=epochs,
    learning_rate=learning_rate,
    # dataset_dict=dataset_dict,
    use_freeze=use_freeze,
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    filename=name_model_ft + '_{epoch:02d}',
    every_n_epochs=1,
    save_top_k=-1,
    enable_version_counter=False, # keep the version == 0
    save_weights_only=True,
)
checkpoint_callback.FILE_EXTENSION = ".pt"

csv_logger = CSVLogger(
    save_dir=dir_model_ft_to_save,
    name=name_model_ft,
    version=0,
)

trainer = L.Trainer(
    default_root_dir=dir_model_ft_to_save,
    # profiler=profiler,
    logger=csv_logger,
    accelerator='auto',
    devices='auto',
    # accelerator='gpu',
    # devices=[0],
    min_epochs=1,
    max_epochs=epochs,
    precision=32,
    callbacks=[checkpoint_callback]
)
trainer.fit(model_ft, data_module)
trainer.validate(model_ft, data_module)

/home/ylee/chemllm/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /opt/software/jupyter-server/2.7.2-GCCcore-12.3.0/l ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ylee/chemllm/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /opt/software/jupyter-server/2.7.2-GCCcore-12.3.0/l ...
/home/ylee/chemllm/lib/python3.11/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory .//save_models_ft/ft_version_0/AbraLlama_solute/version_0 ex

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),
/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),
/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),


Training: |          | 0/? [00:00<?, ?it/s]

/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),
/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),


...

`Trainer.fit` stopped: `max_epochs=7` reached.
/home/ylee/chemllm/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /opt/software/jupyter-server/2.7.2-GCCcore-12.3.0/l ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),
/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.07019355148077011    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.07019355148077011}]

In [7]:

local_model_ft = utils_sol.load_model_ft_with_epoch(
    class_model_ft=model_ft, 
    target_epoch=ep,
    dir_model_ft=dir_model_ft_to_save,
    name_model_ft=name_model_ft
)

result = trainer.predict(local_model_ft, data_module)
result_pred = list()
result_label = list()
for bat in range(len(result)):
    result_pred.append(result[bat][0].squeeze())
    result_label.append(result[bat][1])

/home/ylee/chemllm/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /opt/software/jupyter-server/2.7.2-GCCcore-12.3.0/l ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded model with epoch 6


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),
/home/ylee/SolLlama/datamodule_finetune_sol.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.tensor(self.labels.iloc[idx]),


In [8]:
result_pred[0]

tensor([[ 8.1577e-01,  7.3935e-01,  3.4518e-01,  3.8979e-01,  1.3431e+00],
        [ 2.6664e-01,  1.0558e+00,  2.0594e-02,  7.9018e-01,  9.7986e-01],
        [ 8.0645e-01,  9.9306e-01,  6.4024e-01,  5.7245e-01,  1.2106e+00],
        [ 1.9478e-01,  4.1441e-01,  1.0704e-01,  6.1531e-01,  6.3337e-01],
        [ 2.4553e-01,  4.4279e-01,  3.1961e-01,  5.5282e-01,  8.5223e-01],
        [ 3.1377e-01,  7.8347e-01,  5.6589e-03,  7.1544e-01,  6.5998e-01],
        [ 1.0864e-01,  5.6291e-01,  4.7713e-03,  4.3076e-01,  1.7124e+00],
        [ 1.7752e+00,  2.1543e+00,  1.3406e-02,  1.4107e+00,  1.7843e+00],
        [ 2.0859e+00,  1.5931e+00, -5.9972e-03,  8.9334e-02,  1.9388e+00],
        [ 3.7282e-01,  8.3927e-01,  6.0150e-01,  9.5349e-01,  7.8828e-01],
        [ 1.3070e+00,  2.4475e+00,  2.5248e-01,  1.7554e+00,  3.4275e+00],
        [ 9.9038e-01,  1.3453e+00,  3.1266e-01,  7.1342e-01,  1.6044e+00],
        [ 5.6058e-02,  3.7555e-01,  6.2625e-02,  5.8542e-02,  9.2921e-01],
        [ 1.2678e+00,  1.

In [9]:
result_label[0]

tensor([[ 0.8200,  0.8000,  0.5300,  0.3800,  1.3387],
        [ 0.2600,  1.3100,  0.0600,  0.8100,  0.9854],
        [ 0.7300,  0.9700,  0.6000,  0.6400,  1.2135],
        [ 0.1800,  0.3500,  0.0800,  0.6300,  0.6311],
        [ 0.2900,  0.4500,  0.3000,  0.4100,  0.8288],
        [ 0.3300,  0.7500,  0.0000,  0.6400,  0.6810],
        [ 0.2900,  0.7200,  0.0000,  0.3500,  1.6774],
        [ 1.5000,  2.4100,  0.0000,  1.3400,  1.8016],
        [ 2.1000,  1.5700,  0.0000,  0.0900,  1.9362],
        [ 0.3700,  0.9300,  0.6000,  0.8800,  0.7896],
        [ 1.3500,  2.5500,  0.3200,  1.8600,  3.4268],
        [ 1.0000,  1.3600,  0.2800,  0.7100,  1.5951],
        [ 0.0400,  0.6800,  0.1000,  0.1100,  0.8995],
        [ 1.2600,  1.2700,  0.4000,  0.1300,  1.1834],
        [ 1.6000,  2.4400,  0.0000,  1.0100,  2.4330],
        [ 0.1300,  0.6500,  0.0000,  0.1300,  0.7417],
        [ 0.1100,  0.6500,  0.0000,  0.5100,  0.9697],
        [-0.5300, -0.1600,  0.0000,  0.1100,  0.9225],
        [ 

# dataframe

In [10]:
data_module.test_df['SMILES'].reset_index(drop=True)

0                                CCCc1cc(C)ccc1O
1                                 CC(=O)CCC(=O)C
2                             OC(=O)Cc1ccc(cc1)C
3                                           CNCC
4                                      CC(CC=C)O
                          ...                   
1023                           Ic1cccc(c1)C(=O)C
1024                              CCCCCCCCCCCCCl
1025      Clc1cc(Cl)cc(c1)c1c(Cl)cc(c(c1Cl)Cl)Cl
1026                     CCOC(=O)CCCCCCCC(=O)OCC
1027    CC(=NNC(=O)c1ccncc1)c1cccc(c1O)CN1CCOCC1
Name: SMILES, Length: 1028, dtype: object

In [11]:
list_df_predict = list()
for i in result_pred:
    local_df_predict = pd.DataFrame(np.array(i))
    list_df_predict.append(local_df_predict)

df_predict = pd.concat(list_df_predict)
df_predict.insert(0, "SMILES", data_module.test_df['SMILES'].reset_index(drop=True))
df_predict


,SMILES,0,1,2,3,4
0,CCCc1cc(C)ccc1O,0.815770,0.739354,0.345182,0.389789,1.343119
1,CC(=O)CCC(=O)C,0.266639,1.055772,0.020594,0.790178,0.979856
2,OC(=O)Cc1ccc(cc1)C,0.806450,0.993064,0.640236,0.572451,1.210571
3,CNCC,0.194782,0.414411,0.107041,0.615315,0.633372
4,CC(CC=C)O,0.245530,0.442788,0.319606,0.552818,0.852235
...,...,...,...,...,...,...
63,CCCCCCCC[Si](O[Si](C)(C)C)(O[Si](C)(C)C)C,1.390710,1.284074,0.017194,0.421718,1.258709
0,CCCc1cc(C)ccc1O,0.035455,0.068771,0.000337,-0.000562,1.951705
1,CC(=O)CCC(=O)C,2.183715,1.733424,-0.002669,0.100027,2.073741
2,OC(=O)Cc1ccc(cc1)C,0.075538,1.131518,-0.003029,1.014681,2.092385


In [12]:
df_label = data_module.test_df.reset_index(drop=True)
df_label

,SMILES,label_0,label_1,label_2,label_3,label_4
0,CCCc1cc(C)ccc1O,0.82,0.80,0.53,0.38,1.3387
1,CC(=O)CCC(=O)C,0.26,1.31,0.06,0.81,0.9854
2,OC(=O)Cc1ccc(cc1)C,0.73,0.97,0.60,0.64,1.2135
3,CNCC,0.18,0.35,0.08,0.63,0.6311
4,CC(CC=C)O,0.29,0.45,0.30,0.41,0.8288
...,...,...,...,...,...,...
1023,Ic1cccc(c1)C(=O)C,1.37,1.23,0.00,0.43,1.2721
1024,CCCCCCCCCCCCCl,0.18,0.40,0.00,0.10,1.9218
1025,Clc1cc(Cl)cc(c1)c1c(Cl)cc(c(c1Cl)Cl)Cl,2.19,1.74,0.00,0.11,2.0586
1026,CCOC(=O)CCCCCCCC(=O)OCC,0.06,1.10,0.00,1.02,2.0891
